# Classificação de imagens de leismaniose utilizando CNN pré-treinadas

## GPU

In [1]:
!nvidia-smi

Sun Jun 25 22:36:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:09:00.0 Off |                  N/A |
|  0%   30C    P8     4W / 300W |      5MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:0A:00.0 Off |                  N/A |
|  0%   

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #aqui tem q escolher uma das gpus, veja a que esta desocupada (comando: nvidia-smi)
tf_device='/gpu:0'

## Bibliotecas

In [3]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import NASNetLarge
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.densenet import DenseNet201

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix

import pickle # salvar modelo em disco

from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Clésio Gonçalves" --iversions

Author: Clésio Gonçalves

tensorflow: 2.4.1
numpy     : 1.19.2
matplotlib: 3.3.4
pandas    : 1.2.3



## Parâmetros

In [5]:
# Dados de entrada
PATH_IMAGES = 'dataset/'

# Parâmetros do treinamento
BATCH_SIZE = 10
SEED = 42

# K-fold
N_SPLIT = 5

In [6]:
import shutil

# Remove o diretorio caso exista (antes de criá-lo)
if os.path.exists("modelo") and os.path.isdir("modelo"):
    shutil.rmtree("modelo")
    
# Remove o diretorio caso exista (antes de criá-lo)
if os.path.exists("resultados") and os.path.isdir("resultados"):
    shutil.rmtree("resultados")

In [7]:
!mkdir modelo
!mkdir resultados

In [8]:
# Parâmetros treinamento
epocas_treinamento = 20
base_learning_rate = 0.0001

epocas_ajuste_fino = 10
total_epocas = epocas_treinamento + epocas_ajuste_fino

# Dataset

In [9]:
#CATEGORIAS = ['Monkey Pox', 'Others']
CATEGORIAS = ['Negativo', 'Positivo']
NUM_CATEGORIAS = len(CATEGORIAS)
NUM_CATEGORIAS

2

In [10]:
for category in CATEGORIAS:
    print('{} {} imagens'.format(category, len(os.listdir(os.path.join(PATH_IMAGES, category)))))

Negativo 72 imagens
Positivo 78 imagens


In [11]:
dataset = []
for category in CATEGORIAS:
    for file in os.listdir(os.path.join(PATH_IMAGES, category)):
        dataset.append(['{}/{}'.format(category, file), category])
dataset = pd.DataFrame(dataset, columns=['arquivo', 'categoria'])
dataset.shape

(150, 2)

In [12]:
dataset

,arquivo,categoria
0,Negativo/CM200826-124843056.jpg,Negativo
1,Negativo/CM200826-122443024.jpg,Negativo
2,Negativo/CM200826-122541026.jpg,Negativo
3,Negativo/CM200826-144801011.jpg,Negativo
4,Negativo/CM200826-144527006.jpg,Negativo
...,...,...
145,Positivo/CM200819-105910047.jpg,Positivo
146,Positivo/Imagem 13.jpg,Positivo
147,Positivo/CM200826-100032004.jpg,Positivo
148,Positivo/Imagem 34.jpg,Positivo


In [13]:
# Definindo X e Y
dataset_x = dataset.arquivo
dataset_y = dataset.categoria

# Leitura dos dados

In [14]:
#Initializing Data Generators
datagen = ImageDataGenerator()

In [15]:
# Carrega os dados de treino, validação e testes
def leitura_dados(img_size):
    train_dataset = datagen.flow_from_dataframe(dataframe = train_df, 
                                                      directory = PATH_IMAGES,
                                                      x_col = "arquivo", 
                                                      y_col = "categoria",
                                                      class_mode="binary",
                                                      target_size = img_size, 
                                                      batch_size = BATCH_SIZE,
                                                      seed = SEED,
                                                      shuffle = True)

    val_dataset = datagen.flow_from_dataframe(dataframe = val_df, 
                                                      directory = PATH_IMAGES,
                                                      x_col = "arquivo", 
                                                      y_col = "categoria",
                                                      class_mode="binary",
                                                      target_size = img_size, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = False)

    test_dataset = datagen.flow_from_dataframe(dataframe = test_df, 
                                                      directory = PATH_IMAGES,
                                                      x_col = "arquivo", 
                                                      y_col = "categoria",
                                                      class_mode="binary",
                                                      target_size = img_size, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = False)
    
    return train_dataset, val_dataset, test_dataset

# Métricas

In [16]:
# Cria data frame de resultados
colunas_dataframe = ['model', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'kappa', 'precision', 'f1score', 'recall', 'specificity', 'roc_auc']
resultados = pd.DataFrame(columns = colunas_dataframe)

def calcula_metricas_binarias(y_true, y_pred):
    
    global resultados
    
    # Cutoff
    y_true = (y_true > 0.5).flatten()
    y_pred = (y_pred > 0.5).flatten()

    cm = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = accuracy_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = (1.0 * tn) / (tn + fp)
    roc_auc = roc_auc_score(y_true, y_pred)
    
    metricas = {'model': models[rede].name, 
                'tp': tp, 
                'fp': fp,
                'tn': tn,
                'fn': fn,
                'accuracy': accuracy,
                'kappa': kappa,
                'precision': precision,
                'f1score': f1score,
                'recall': recall,
                'specificity': specificity,
                'roc_auc':roc_auc}
    
    print(metricas)
    
    resultados = resultados.append(metricas, ignore_index=True)

# Modelo

In [17]:
# leitura dos pesos pré-treinados da ImageNet
# Não inclui as camadas de classificação no topo, ideal para extração de features
models = [
    DenseNet201(weights = "imagenet", input_shape = (224, 224, 3), include_top = False),
    InceptionV3(weights='imagenet', input_shape=(299, 299, 3), include_top=False),
    Xception(weights='imagenet', input_shape=(299, 299, 3), include_top=False),
    InceptionResNetV2(weights='imagenet', input_shape=(299, 299, 3), include_top=False),
    NASNetLarge(weights='imagenet', input_shape=(331, 331, 3), include_top=False),
    ResNet152V2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
]

# Reescala dos valores dos pixels do modelo
processamento_input = [
    keras.applications.densenet.preprocess_input,
    keras.applications.inception_v3.preprocess_input,
    keras.applications.xception.preprocess_input,
    keras.applications.inception_resnet_v2.preprocess_input,
    keras.applications.nasnet.preprocess_input,
    keras.applications.resnet_v2.preprocess_input
]

In [18]:
# Construir um modelo a partir de redes neurais pré-treinadas
def modelo_base(rede):
    
    base_model = models[rede]
    img_size = (base_model.input.shape[1], base_model.input.shape[2])
    
    # Processa as entradas do modelo
    preprocess_input = processamento_input[rede]
    
    return base_model, img_size, preprocess_input

In [19]:
# Usando aumento de dados aleatórios somente no fit (treinamento)
data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        #keras.layers.experimental.preprocessing.RandomRotation(0.9), 
        keras.layers.experimental.preprocessing.RandomZoom(0.1),
        keras.layers.experimental.preprocessing.RandomContrast(0.2)
    ]
)

In [20]:
# Construir modelo
def build_model():
    
    # Freeze the base_model
    # Congela a base convolucional antes de compilar e treinar o modelo
    # Evita que os pesos em uma determinada camada sejam atualizados durante o treinamento
    base_model.trainable = False
    
    # Arquitetura do modelo básico
    # base_model.summary()

    # Adiciona o cabeçalho de classificação
    # Converter as features do shape `base_model.output_shape[1:] para vectores
    global_average_layer = keras.layers.GlobalAveragePooling2D()

    # Aplica uma camada densa para converter essas features em uma única previsão por imagem
    # Os números > 0.5 preveem a classe 1, os números <= 0.5 preveem a classe 0
    prediction_layer = keras.layers.Dense(1, activation="sigmoid", name='predictions') # Função de ativação sigmoid adicionada

    # Modelo encadeando as camadas de aumento de dados, reescalonamento, base_model e extrator de features
    img_shape = img_size + (3,)
    inputs = keras.Input(shape = img_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = keras.Model(inputs, outputs)
    
    return model

In [21]:
# Compile o modelo antes de treiná-lo
def compile_model(learning_rate):
    
    # Compilar modelo
    # Não especifiquei o batch_size, pois os dados já estão em conjuntos (batchs)
    model.compile(optimizer = Adam(lr = learning_rate), loss = keras.losses.BinaryCrossentropy(), metrics = ['binary_accuracy'])
    
    # model.summary()

In [22]:
# Definimos um checkpoint para verificar regularmente se a perda em validação diminuiu
# Se a performance melhorar em validação salvamos o modelo
# Podemos ainda optar por salvar o modelo a cada número de épocas
# callbacks
# Redução gradual da taxa de aprendizado (Reduce on Plateau)
def get_callbacks():
    return [EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1),
            ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 5, min_lr = 0.0000001, verbose = 1),
            ModelCheckpoint('modelo/{}.h5'.format(models[rede].name), 
                         verbose = 1, 
                         save_best_only = True, 
                         save_weights_only = True)]

In [23]:
# Salva o modelo
def salva_estrutura_modelo():
    
    # salva o modelo em disco
    # pickle.dump(model, open(f'modelo/{models[rede].name}.pkl', 'wb'))
    model.save(f'modelo/{models[rede].name}')
    
    # salva json
    arquivo_modelo = f'modelo/{models[rede].name}.json'
    modelo_json = model.to_json()
    with open(arquivo_modelo, 'w') as json_file:
        json_file.write(modelo_json)

In [24]:
# Treinamento do modelo
def treinamento_model(qnt_epocas, epoca_inicial):
    
    history = model.fit(train_dataset,
                        epochs = qnt_epocas,
                        initial_epoch = epoca_inicial,
                        validation_data = val_dataset,
                        verbose=1,
                        callbacks = get_callbacks())
    
    return history

In [25]:
# Curvas de aprendizado da precisão / perda de treinamento e validação ao usar o modelo
def aprendizado_treinamento():
    
    metricas_graficos = ["loss", "binary_accuracy"]
          
    fig, ax = plt.subplots(len(metricas_graficos), 1, figsize=(10, len(metricas_graficos)*6))
    ax = ax.ravel()
    dados_x = np.arange(1, epocas_treinamento+1, 1)

    for i, met in enumerate(metricas_graficos):        
        ax[i].plot(dados_x, history.history[met], label='Training ' + met)
        ax[i].plot(dados_x, history.history["val_" + met], label='Validation ' + met)
        ax[i].set_title("Training and Validation %s in model %s" %(met, models[rede].name))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(met)
        
        if (i != 0): # loss function
            ax[i].legend(loc='lower right')
        else:
            ax[i].legend(loc='upper right')
    
    return metricas_graficos

In [26]:
# LIMIAR PARA AJUSTE FINO (OPCIONAL)
# Definir as camadas inferiores como não treináveis
def limiar_ajuste_fino():
    
    # Exibe a quantidade de camadas do modelo base
    # print("Número de camadas no modelo base: ", len(base_model.layers))

    # Ajuste fino desta camada em diante
    fine_tune_at = 100

    # Congele todas as camadas antes da camada 'fine_tune_at'
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

In [27]:
# Ajuste fino
def ajuste_fino():
    
    # Foi treinado apenas algumas camadas do modelo. 
    # Os pesos da rede pré-treinada não foram atualizados durante o treinamento.
    # Uma maneira de aumentar ainda mais o desempenho é treinar (ou "ajustar") os pesos das camadas superiores 
    # do modelo pré-treinado junto com o treinamento do classificador adicionado (camada de classificação adicionada)
    # Descongelar as camadas superiores do modelo (descongelar base_model)
    base_model.trainable = True
    
    # Limiar ajuste fino (OPCIONAL)
    limiar_ajuste_fino()
    
    # É necessário recompilar o modelo (para que essas alterações tenham efeito)
    # É importante usar uma taxa de aprendizado mais baixa neste estágio, 
    # pois está usando um modelo muito maior e deseja readaptar os pesos pré-treinados
    compile_model(base_learning_rate/10)
    
    # Retomar o treinamento melhorará sua precisão em alguns pontos percentuais
    # history.epoch[-1] é a última época do ultimo treinamento
    history_fine = treinamento_model(total_epocas, history.epoch[-1]+1)
    
    return history_fine

In [28]:
# Curvas de aprendizado da precisão / perda de treinamento e validação ao ajustar as últimas camadas do modelo
def aprendizado_ajuste_fino():
    
    fig, ax = plt.subplots(len(metricas_graficos), 1, figsize=(12, len(metricas_graficos)*6))
    ax = ax.ravel()
    dados_x = np.arange(1, len(history.history['loss']) + len(history_fine.history['loss'])+1, 1)

    for i, met in enumerate(metricas_graficos):
        dados_treino = history.history[met] + history_fine.history[met]
        dados_validacao = history.history["val_" + met] + history_fine.history["val_" + met]
        
        ax[i].plot(dados_x, dados_treino, label='Training ' + met)
        ax[i].plot(dados_x, dados_validacao, label='Validation ' + met)
        ax[i].plot([epocas_treinamento, epocas_treinamento], plt.ylim(), label='Start Fine Tuning')
        ax[i].set_title("Training and Validation %s in model %s" %(met, models[rede].name))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(met)
        
        if (i != 0): # loss function
            ax[i].legend(loc='lower right')
        else:
            ax[i].legend(loc='upper right')

# Treinamento K-fold

In [29]:
%%time

# k-fold
kfold = StratifiedKFold(n_splits = N_SPLIT, shuffle = True, random_state = SEED)

# Contador de iterações do k-fold
iteracao = 1

# K-fold treino e teste de cada divisão
for train_idx, test_idx in list(kfold.split(dataset_x, dataset_y)):
    
    print("\n======================================================")
    print("Iteração {} de {}".format(iteracao, N_SPLIT))
    print("======================================================")
    
    trein_temp = dataset.iloc[train_idx]
    test_df = dataset.iloc[test_idx] # 20% teste
    
    # dividir o teste em validação e teste
    train_df, val_df = train_test_split(trein_temp, test_size = 0.13, random_state = SEED) # 10% validação e 70% treino
    
    for rede in range(len(models)): # Todos os modelos
        
        print('\nExecutando modelo {}'.format(models[rede].name))

        # Modelo base
        base_model, img_size, preprocess_input = modelo_base(rede)

        # Leitura dos dados
        train_dataset, val_dataset, test_dataset = leitura_dados(img_size)

        # Construir modelo
        model = build_model()

        # Compilar Modelo
        compile_model(base_learning_rate)

        # Treinamento do modelo
        history = treinamento_model(epocas_treinamento, 0)

        # Curvas de aprendizado da precisão / perda de treinamento e validação ao usar o modelo
        # metricas_graficos = aprendizado_treinamento()

        # Ajuste fino
        history_fine = ajuste_fino()

        # Curvas de aprendizado da precisão / perda de treinamento e validação ao ajustar as últimas camadas do modelo
        # aprendizado_ajuste_fino()
        
        # Carrega o melhor modelo
        model.load_weights('modelo/{}.h5'.format(models[rede].name))
        
        # Salva a estrutura do modelo
        salva_estrutura_modelo()

        # Obtemos os rótulos verdadeiros
        y_true = np.array(test_dataset.classes)

        # Obtemos os rótulos previstos
        y_pred = model.predict(test_dataset, verbose = 1)
        # y_pred = previsoes.argmax(axis=1)

        # Calcula métricas Binárias
        calcula_metricas_binarias(y_true, y_pred)

        # Limpa a sessão
        keras.backend.clear_session()

    iteracao = iteracao + 1
    break


Iteração 1 de 5

Executando modelo densenet201
Found 104 validated image filenames belonging to 2 classes.
Found 16 validated image filenames belonging to 2 classes.
Found 30 validated image filenames belonging to 2 classes.
Epoch 1/20
11/11 [==============================] - 13s 563ms/step - loss: 0.9007 - binary_accuracy: 0.3047 - val_loss: 0.7467 - val_binary_accuracy: 0.4375

Epoch 00001: val_loss improved from inf to 0.74668, saving model to modelo/densenet201.h5
Epoch 2/20
11/11 [==============================] - 3s 244ms/step - loss: 0.8336 - binary_accuracy: 0.3684 - val_loss: 0.7273 - val_binary_accuracy: 0.4375

Epoch 00002: val_loss improved from 0.74668 to 0.72726, saving model to modelo/densenet201.h5
Epoch 3/20
11/11 [==============================] - 3s 248ms/step - loss: 0.7571 - binary_accuracy: 0.5443 - val_loss: 0.7136 - val_binary_accuracy: 0.4375

Epoch 00003: val_loss improved from 0.72726 to 0.71359, saving model to modelo/densenet201.h5
Epoch 4/20
11/11 [======

In [30]:
resultados

,model,tp,fp,tn,fn,accuracy,kappa,precision,f1score,recall,specificity,roc_auc
0,densenet201,14,0,15,1,0.966667,0.933333,1.000000,0.965517,0.933333,1.000000,0.966667
1,inception_v3,12,1,14,3,0.866667,0.733333,0.923077,0.857143,0.800000,0.933333,0.866667
2,xception,12,0,15,3,0.900000,0.800000,1.000000,0.888889,0.800000,1.000000,0.900000
3,inception_resnet_v2,14,0,15,1,0.966667,0.933333,1.000000,0.965517,0.933333,1.000000,0.966667
4,NASNet,11,0,15,4,0.866667,0.733333,1.000000,0.846154,0.733333,1.000000,0.866667
5,resnet152v2,11,0,15,4,0.866667,0.733333,1.000000,0.846154,0.733333,1.000000,0.866667


In [31]:
# Resultados de todas as iterações
resultados.to_csv('resultados/resultados.csv', index=False, header=True)